In [1]:
import pandas as pd
import re
import censusdata
from pprint import pprint


pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 2)
pd.options.display.max_columns = None

from scripts.FetchCensusCodes import fetch_variables_and_labels, fetch_data_frame, IL_metro_county_codes, MO_metro_county_codes, survey, year
from resources.api_keys import census_api_key

In [2]:
# censusdata.printtable(censusdata.censustable(survey, year, table))

In [3]:
# states = censusdata.geographies(censusdata.censusgeo([('state', '*')]), survey, year)
# pprint(states)

In [4]:
STL_geo_codes = []

for county in MO_metro_county_codes:
    STL_geo_codes.append(('29', county))

for county in IL_metro_county_codes:
    STL_geo_codes.append(('17', county))

In [5]:
MOcounties = censusdata.geographies(censusdata.censusgeo([('state', '29'), ('county', '*')]), survey, year)
ILcounties = censusdata.geographies(censusdata.censusgeo([('state', '17'), ('county', '*')]), survey, year)

MO_geo_codes = []
IL_geo_codes = []

for key in MOcounties:
    statecode = str(MOcounties[key])[26:28]
    countycode = str(MOcounties[key])[-3:]
    MO_geo_codes.append((statecode, countycode))

for key in ILcounties:
    statecode = str(ILcounties[key])[26:28]
    countycode = str(ILcounties[key])[-3:]
    IL_geo_codes.append((statecode, countycode))

In [6]:
STL_FIPS = []
MO_FIPS = []
IL_FIPS = []

for codes in STL_geo_codes:
    STL_FIPS.append(codes[0]+codes[1])

for codes in MO_geo_codes:
    MO_FIPS.append(codes[0]+codes[1])

for codes in IL_geo_codes:
    IL_FIPS.append(codes[0]+codes[1])

## Industry By Median Earnings In The Past 12 Months (In 2018 Inflation-Adjusted Dollars) For The Civilian Employed Population 16 Years And Over (B24041)##

In [7]:
# STL_industry_by_median_earning = fetch_data_frame('B24041', STL_geo_codes)

In [8]:
MO_industry_by_median_earning = fetch_data_frame('B24041', MO_geo_codes)
IL_industry_by_median_earning = fetch_data_frame('B24041', IL_geo_codes)

In [9]:
essential_industries = ['Retail trade', 'Transportation and warehousing, and utilities', 'Administrative and support and waste management services', 'Educational services, and health care and social assistance', 'Accommodation and food services']

In [10]:
combined_median_earnings = pd.concat([MO_industry_by_median_earning, IL_industry_by_median_earning])

In [11]:
essential_median_earnings = combined_median_earnings[essential_industries]

In [12]:
essential_median_earnings = essential_median_earnings.reset_index()
essential_median_earnings['index'] = essential_median_earnings['index'].astype(str)

In [13]:
essential_median_earnings['statecode'] = essential_median_earnings['index'].str.extract(r'((?<=state:)[0-9]{2})')
essential_median_earnings['countycode'] = essential_median_earnings['index'].str.extract(r'((?<=county:)[0-9]{3})')
essential_median_earnings['GeoFips'] = essential_median_earnings['statecode'] + essential_median_earnings['countycode']

In [14]:
essential_median_earnings_final = essential_median_earnings[['GeoFips'] + essential_industries].copy()

In [15]:
essential_median_earnings_final['GeoFips'] = essential_median_earnings_final['GeoFips'].astype('int64')
essential_median_earnings_final.loc[essential_median_earnings_final['GeoFips'].isin((MO_FIPS + IL_FIPS))].reset_index(drop = True)

,GeoFips,Retail trade,"Transportation and warehousing, and utilities",Administrative and support and waste management services,"Educational services, and health care and social assistance",Accommodation and food services
0,29195,30534,60214,22318,29471,22348
1,29123,26321,63648,24688,30315,31818
2,29229,22773,30096,25714,26946,18167
3,29217,25332,54539,20224,35367,27750
4,29147,24167,55156,20625,38243,17898
...,...,...,...,...,...,...
212,17019,34668,50642,37756,50027,26109
213,17053,29479,57135,30278,44650,29519
214,17093,41549,63072,49328,54337,38547
215,17121,32096,54744,45526,38763,21078


In [16]:
# essential_median_earnings_final

,GeoFips,Retail trade,"Transportation and warehousing, and utilities",Administrative and support and waste management services,"Educational services, and health care and social assistance",Accommodation and food services
0,29195,30534,60214,22318,29471,22348
1,29123,26321,63648,24688,30315,31818
2,29229,22773,30096,25714,26946,18167
3,29217,25332,54539,20224,35367,27750
4,29147,24167,55156,20625,38243,17898
...,...,...,...,...,...,...
212,17019,34668,50642,37756,50027,26109
213,17053,29479,57135,30278,44650,29519
214,17093,41549,63072,49328,54337,38547
215,17121,32096,54744,45526,38763,21078


## EMPLOYMENT STATUS FOR THE POPULATION 16 YEARS AND OVER (B23025)##

### MO

In [17]:
fetch_MO_employment = fetch_data_frame('B23025', MO_geo_codes)

In [18]:
MO_employment = fetch_MO_employment[['In labor force', 'Employed', 'Unemployed']]
MO_employment['Unemployment Rate'] = ((MO_employment['Unemployed'] / MO_employment['In labor force']) * 100).apply(lambda x: round(x, 2))

<ipython-input-18-53a6af9f770b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MO_employment['Unemployment Rate'] = ((MO_employment['Unemployed'] / MO_employment['In labor force']) * 100).apply(lambda x: round(x, 2))


In [19]:
MO_employment = MO_employment.reset_index()
MO_employment['index'] = MO_employment['index'].astype(str)
MO_employment['statecode'] = MO_employment['index'].str.extract(r'((?<=state:)[0-9]{2})')
MO_employment['countycode'] = MO_employment['index'].str.extract(r'((?<=county:)[0-9]{3})')
MO_employment['GeoFips'] = MO_employment['statecode'] + MO_employment['countycode']

In [20]:
MO_employment_final = MO_employment[['GeoFips', 'In labor force', 'Unemployment Rate']].copy()

In [21]:
# MO_employment_final

,GeoFips,In labor force,Unemployment Rate
0,29195,10955,3.94
1,29123,5274,4.11
2,29229,7280,8.04
3,29217,9920,3.67
4,29147,11515,3.85
...,...,...,...
110,29155,6763,9.79
111,29203,3099,6.32
112,29067,5118,6.17
113,29083,9473,6.43


### IL

In [22]:
fetch_IL_employment = fetch_data_frame('B23025', IL_geo_codes)

In [23]:
IL_employment = fetch_IL_employment[['In labor force', 'Employed', 'Unemployed']]
IL_employment['Unemployment Rate'] = ((IL_employment['Unemployed'] / IL_employment['In labor force']) * 100).apply(lambda x: round(x, 2))

<ipython-input-23-3e6847d2a8b7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  IL_employment['Unemployment Rate'] = ((IL_employment['Unemployed'] / IL_employment['In labor force']) * 100).apply(lambda x: round(x, 2))


In [24]:
IL_employment = IL_employment.reset_index()
IL_employment['index'] = IL_employment['index'].astype(str)
IL_employment['statecode'] = IL_employment['index'].str.extract(r'((?<=state:)[0-9]{2})')
IL_employment['countycode'] = IL_employment['index'].str.extract(r'((?<=county:)[0-9]{3})')
IL_employment['GeoFips'] = IL_employment['statecode'] + IL_employment['countycode']

In [25]:
IL_employment_final = IL_employment[['GeoFips', 'In labor force', 'Unemployment Rate']].copy()

In [26]:
# IL_employment_final

,GeoFips,In labor force,Unemployment Rate
0,17083,11127,4.05
1,17155,3107,3.60
2,17039,8614,5.48
3,17051,9274,6.98
4,17103,16172,4.40
...,...,...,...
97,17019,108838,4.78
98,17053,6618,5.33
99,17093,67387,4.85
100,17121,18482,9.45


### combined

In [27]:
combined_employment = pd.concat([MO_employment_final, IL_employment_final])

In [28]:
combined_employment

,GeoFips,In labor force,Unemployment Rate
0,29195,10955,3.94
1,29123,5274,4.11
2,29229,7280,8.04
3,29217,9920,3.67
4,29147,11515,3.85
...,...,...,...
97,17019,108838,4.78
98,17053,6618,5.33
99,17093,67387,4.85
100,17121,18482,9.45


# GPD #

### MO

In [29]:
MO_GDP = pd.read_csv('./data/GDP/GDP_by_county_MO.csv')

In [30]:
MO_GDP = MO_GDP.loc[MO_GDP['GeoFips'].isin(MO_FIPS)].reset_index(drop = True)

In [31]:
# MO_GDP

,GeoFips,GDP (Thousands of dollars)
0,29001,863728
1,29003,318311
2,29005,277298
3,29007,1012363
4,29009,1363419
...,...,...
110,29223,244420
111,29225,756505
112,29227,65222
113,29229,417381


### IL

In [32]:
IL_GDP = pd.read_csv('./data/GDP/GDP_by_county_IL.csv')

In [33]:
IL_GDP = IL_GDP.loc[IL_GDP['GeoFips'].isin(IL_FIPS)].reset_index(drop = True)

In [34]:
# IL_GDP

,GeoFips,GDP (Thousands of dollars)
0,17001,3676937
1,17003,199171
2,17005,617298
3,17007,1931882
4,17009,445261
...,...,...
97,17195,2330100
98,17197,31475135
99,17199,3105878
100,17201,14817252


### combined

In [35]:
combined_GDP = pd.concat([MO_GDP, IL_GDP])
combined_employment['GeoFips'] = combined_employment['GeoFips'].astype('int64')

In [36]:
combined_GDP

,GeoFips,GDP (Thousands of dollars)
0,29001,863728
1,29003,318311
2,29005,277298
3,29007,1012363
4,29009,1363419
...,...,...
97,17195,2330100
98,17197,31475135
99,17199,3105878
100,17201,14817252


## Essential Employment

In [45]:
essential_employment = pd.read_csv('./data/employment/employment_all.csv')

In [48]:
essential_employment = essential_employment.loc[essential_employment['GeoFips'].isin(MO_FIPS + IL_FIPS)].reset_index(drop = True)

In [54]:
essential_employment_final = essential_employment[['GeoFips', 'frontline']].copy()

In [57]:
essential_employment_final['GeoFips'] = essential_employment['GeoFips'].astype('int64')

In [58]:
essential_employment_final

,GeoFips,frontline
0,29001,3201
1,29003,985
2,29005,540
3,29007,3077
4,29009,3962
...,...,...
212,17195,8608
213,17197,127440
214,17199,11137
215,17201,52236


## Join it all together

In [72]:
combined_employment_final = combined_employment.set_index('GeoFips').join(essential_employment_final.set_index('GeoFips'))

In [73]:
combined_employment_final['Frontline Industry Rate'] = ((combined_employment_final['frontline'] / combined_employment_final['In labor force']) * 100).apply(lambda x: round(x, 2))

In [78]:
combined_employment_final = pd.DataFrame(combined_employment_final[['In labor force', 'Frontline Industry Rate', 'Unemployment Rate']])

In [85]:
combined_employment_final = combined_employment_final.reset_index()

In [87]:
combined_employment_final['GeoFips'] = combined_employment_final['GeoFips'].astype('int64')

In [88]:
combined_employment_final

,GeoFips,In labor force,Frontline Industry Rate,Unemployment Rate
0,29195,10955,17.06,3.94
1,29123,5274,23.21,4.11
2,29229,7280,22.45,8.04
3,29217,9920,13.77,3.67
4,29147,11515,24.18,3.85
...,...,...,...,...
212,17019,108838,30.64,4.78
213,17053,6618,19.75,5.33
214,17093,67387,24.45,4.85
215,17121,18482,25.59,9.45


In [65]:
shapefile = combined_GDP.set_index('GeoFips').join(combined_employment.set_index('GeoFips').join(essential_median_earnings_final.set_index('GeoFips')))
shapefile = shapefile.reset_index()

In [66]:
shapefile

,GeoFips,GDP (Thousands of dollars),In labor force,Unemployment Rate,Retail trade,"Transportation and warehousing, and utilities",Administrative and support and waste management services,"Educational services, and health care and social assistance",Accommodation and food services,frontline
0,29001,863728,11611,6.47,26500,39519,26758,40884,16667,3201
1,29003,318311,8780,3.14,31821,47188,26250,41347,22008,985
2,29005,277298,2739,3.29,27083,62500,33906,37019,17917,540
3,29007,1012363,11578,7.14,29154,33750,24643,32336,20403,3077
4,29009,1363419,15311,4.71,27215,41816,41118,31953,24188,3962
...,...,...,...,...,...,...,...,...,...,...
212,17195,2330100,27880,5.28,37078,53410,39263,36185,20917,8608
213,17197,31475135,367773,5.15,37922,59451,38823,54828,29934,127440
214,17199,3105878,30742,5.87,34530,51968,27377,46319,21899,11137
215,17201,14817252,146107,9.11,32010,52218,28091,44789,26731,52236


In [42]:
shapefile.to_csv('./resources/county_shapefile.csv', index=False)